In [ ]:

import os
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Asegurarse de que las gráficas se muestren en el notebook
%matplotlib inline

# Ajustar el valor de agg.path.chunksize para evitar el OverflowError
import matplotlib as mpl
mpl.rcParams['agg.path.chunksize'] = 10000

# Añadir la ruta base del proyecto a sys.path
base_path = os.path.abspath('..')
if base_path not in sys.path:
    sys.path.append(base_path)

# Importar librerías
try:
    from src.data.data_loader import load_and_process_data
    from src.strategies.base_strategy import MovingAverageStrategy
    from src.utils.logger import Logger
    print("Importaciones realizadas correctamente")
except ModuleNotFoundError as e:
    print(f"Error en las importaciones: {e}")
    raise

# Verificar rutas de configuración y datos de mercado
data_path = os.path.abspath('../src/data/data.csv')

if not os.path.exists(data_path):
    print(f"El archivo de datos no existe: {data_path}")

# Cargar datos de mercado
try:
    logger = Logger(__name__)
    market_data = load_and_process_data(data_path)
    if market_data is not None:
        print("Datos de mercado cargados correctamente.")
        print(market_data.head())

        # Calcular el precio de cierre como el promedio de Bid y Ask
        market_data['close'] = (market_data['Bid'] + market_data['Ask']) / 2

        # Crear una instancia de la estrategia y obtener señales
        strategy = MovingAverageStrategy(short_window=40, long_window=100)
        signals = strategy.get_signal(market_data)

        # Visualizar el gráfico de medias móviles
        plt.figure(figsize=(12, 6))
        plt.bar(signals.index, signals['close'], label='Price', color='gray', alpha=0.3)
        plt.plot(signals.index, signals['short_mavg'], label='Short Moving Average', color='blue')
        plt.plot(signals.index, signals['long_mavg'], label='Long Moving Average', color='red')
        plt.title('EUR/USD Price Chart with Moving Averages')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.legend(loc='upper left')  # Especificar una ubicación fija para la leyenda
        plt.grid(True)
        plt.show()

        # Predicción de precios usando scikit-learn
        # Preparar los datos para la predicción
        X = market_data[['Bid', 'Ask', 'Volume']]
        y = market_data['close']

        # Dividir los datos en conjuntos de entrenamiento y prueba
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Entrenar un modelo de regresión lineal
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Hacer predicciones
        predictions = model.predict(X_test)

        # Visualizar las predicciones
        plt.figure(figsize=(12, 6))
        plt.plot(y_test.index, y_test, label='Actual', color='blue', linestyle='-', marker='o')
        plt.plot(y_test.index, predictions, label='Predicted', color='red', linestyle='--', marker='x')
        plt.title('Predicción de Precios de Cierre')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.legend(loc='upper left')  # Especificar una ubicación fija para la leyenda
        plt.grid(True)
        plt.show()

    else:
        logger.error("No se pudieron cargar los datos")

except Exception as e:
    logger.error(f"Error al cargar los datos del mercado: {e}")

In [1]:


import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import dask.dataframe as dd
import logging

# Asegurarse de que las gráficas se muestren en el notebook
%matplotlib inline

# Ajustar el valor de agg.path.chunksize para evitar el OverflowError
import matplotlib as mpl
mpl.rcParams['agg.path.chunksize'] = 10000

# Configuración de los logs
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

def load_and_process_data(file_path):
    try:
        logger.info(f"Cargando y procesando datos desde {file_path} usando dask...")

        # Leer el archivo CSV usando dask
        ddf = dd.read_csv(
            file_path,
            header=0,  # Asumimos que el archivo CSV tiene encabezados
            names=["DateTime", "Bid", "Ask", "Volume"],
            dtype={"DateTime": str, "Bid": float, "Ask": float, "Volume": int}  # Especificar tipos de datos
        )

        # Convertir la columna 'DateTime' a tipo datetime
        ddf['DateTime'] = dd.to_datetime(ddf['DateTime'], format='%Y%m%d %H:%M:%S.%f', errors='coerce')

        # Filtrar filas con valores inválidos o NaN
        ddf = ddf.dropna()

        # Convertir el DataFrame de dask a pandas
        all_data = ddf.compute()

        # Configurar la columna 'DateTime' como índice
        all_data.set_index('DateTime', inplace=True)

        logger.info("Datos cargados y procesados correctamente.")
        return all_data

    except Exception as e:
        logger.error(f"Error al procesar los datos: {e}")
        return None

# Ruta al archivo CSV
data_path = '../src/data/data.csv'

# Cargar datos de mercado
try:
    logger = Logger(__name__)
    market_data = load_and_process_data(data_path)
    if market_data is not None:
        print("Datos de mercado cargados correctamente.")
        print(market_data.head())

        # Calcular el precio de cierre como el promedio de Bid y Ask
        market_data['close'] = (market_data['Bid'] + market_data['Ask']) / 2

        # Crear una instancia de la estrategia y obtener señales
        strategy = MovingAverageStrategy(short_window=40, long_window=100)
        signals = strategy.get_signal(market_data)

        # Visualizar el gráfico de medias móviles
        plt.figure(figsize=(12, 6))
        plt.bar(signals.index, signals['close'], label='Price', color='gray', alpha=0.3)
        plt.plot(signals.index, signals['short_mavg'], label='Short Moving Average', color='blue')
        plt.plot(signals.index, signals['long_mavg'], label='Long Moving Average', color='red')
        plt.title('EUR/USD Price Chart with Moving Averages')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.legend(loc='upper left')  # Especificar una ubicación fija para la leyenda
        plt.grid(True)
        plt.show()

        # Predicción de precios usando scikit-learn
        # Preparar los datos para la predicción
        X = market_data[['Bid', 'Ask', 'Volume']]
        y = market_data['close']

        # Dividir los datos en conjuntos de entrenamiento y prueba
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Entrenar un modelo de regresión lineal
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Hacer predicciones
        predictions = model.predict(X_test)

        # Visualizar las predicciones
        plt.figure(figsize=(12, 6))
        plt.plot(y_test.index, y_test, label='Actual', color='blue', linestyle='-', marker='o')
        plt.plot(y_test.index, predictions, label='Predicted', color='red', linestyle='--', marker='x')
        plt.title('Predicción de Precios de Cierre')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.legend(loc='upper left')  # Especificar una ubicación fija para la leyenda
        plt.grid(True)
        plt.show()

    else:
        logger.error("No se pudieron cargar los datos")

except Exception as e:
    logger.error(f"Error al cargar los datos del mercado: {e}")

2025-01-20 11:27:27,909 - ERROR - Error al cargar los datos del mercado: name 'Logger' is not defined
